In [78]:
from py2neo import Graph
from dotenv import load_dotenv
import os

# Load environment variables
load_dotenv()
neo4j_password = os.getenv('NEO4J_PASSWORD')

In [79]:
graph = Graph("neo4j://localhost:7687", auth=("neo4j", neo4j_password))

In [80]:
def get_connected_nodes(node_name):
    # Define the Cypher query to find connected nodes and return both Name and Desc properties
    query = """
    MATCH (n {Name: $node_name})-[:PARENT_OF]-(connected_nodes)
    RETURN connected_nodes.Name AS name, connected_nodes.Desc AS desc
    """
    
    # Execute the query
    results = graph.run(query, node_name=node_name)
    
    # Extract the Name and Desc of each connected node
    connected_nodes_info = []
    for record in results:
        name = record["name"]
        desc = record["desc"]
        connected_nodes_info.append({"Name": name, "Desc": desc})
    
    return connected_nodes_info

In [81]:
from dotenv import load_dotenv
import os 
load_dotenv('.env')

True

In [82]:
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.prompts import ChatPromptTemplate


In [83]:
from langchain.output_parsers import ResponseSchema, StructuredOutputParser

response_schemas = [
    ResponseSchema(name="current_output", description="Output from the current agent that will be passed to the next agent. If there is no next agent, put None"),
    ResponseSchema(name="next_agent", description="The next agent to handle the request."),
    ResponseSchema(name="next_task", description="The task or action for the next agent."),
]

output_parser = StructuredOutputParser.from_response_schemas(response_schemas)


In [84]:
format_instructions = output_parser.get_format_instructions()

In [90]:
search_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """You are a highly skilled assistant designed to search and answer user's general queries
            Given a list of connected agents with their names and description, if required return the name of the next agent you require for your assistance as well as give an instruction for it.
               List of agents : {Connected_agents}\n
               Format Instructions : {format_instructions}\n
               Previous Context : {current_output}"""

        ),
        ("placeholder", "{chat_history}"),
        ("human", "{input}"),   
        ("placeholder", "{agent_scratchpad}"),

    ]
)

greeting_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """You are a friendly AI that greets people warmly when they start a conversation and says goodbye appropriately when they mention leaving. 

                If someone says something about going away, respond with a proper goodbye.
                If they just greet you or talk normally, respond with a friendly greeting.
                Given a list of connected agents with their names and description, if required return the name of the next agent you require for your assistance as well as give an instruction for it.
               List of agents : {Connected_agents} \n
               Format Instructions : {format_instructions}\n
               Previous Context : {current_output}"""
        ),
        ("placeholder", "{chat_history}"),
        ("human", "{input}"),
        ("placeholder", "{agent_scratchpad}"),
    ]
)


math_prompt=ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """You are a highly skilled assistant with expertise in solving math problems. Ensure accurate calculations and provide clear explanations for mathematical concepts. Use the tavily_search_results_json tool for additional information if needed.
                Given a list of connected agents with their names and description, if required return the name of the next agent you require for your assistance as well as give an instruction for it.
               List of agents : {Connected_agents} \n
               Format Instructions : {format_instructions}\n
               Previous Context : {current_output}"""
        ),
        ("placeholder", "{chat_history}"),
        ("human", "{input}"),
        ("placeholder", "{agent_scratchpad}"),
    ]
)


tool_prompt=ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """You are an AI assistant with access to powerful tools for calculations. Use these calculation tools when you need to perform precise calculations. 
               Given a list of connected agents with their names and description, if required return the name of the next agent you require for your assistance as well as give an instruction for it.
               List of agents : {Connected_agents} \n
               Format Instructions : {format_instructions}\n
               Previous Context : {current_output}"""
        ),
        ("placeholder", "{chat_history}"),
        ("human", "{input}"),
        ("placeholder", "{agent_scratchpad}"),
    ]
)

supervisor_prompt=ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """You are an AI assistant who is the first agent tasked to find the appropriate next agent given the task. 
               Given a list of connected agents with their names and description, if required return the name of the next agent you require for your assistance as well as give an instruction for it.
               List of agents : {Connected_agents} \n
               Format Instructions : {format_instructions}"""
        ),
        ("placeholder", "{chat_history}"),
        ("human", "{input}"),
        ("placeholder", "{agent_scratchpad}"),
    ]
)

In [91]:
import os
import openai

OPENAI_API_KEY = os.environ.get('OPENAI_API_KEY')
openai.api_key = OPENAI_API_KEY
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY
TAVILY_API_KEY = os.getenv('TAVILY_API_KEY')
LANGSMITH_API_KEY = os.getenv('LANGSMITH_API_KEY')


In [92]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0, openai_api_key=OPENAI_API_KEY)

In [93]:
search = TavilySearchResults(tavily_api_key = TAVILY_API_KEY)
tools = [search]

In [94]:
from langchain.agents import create_tool_calling_agent

agent2 = create_tool_calling_agent(llm, tools, search_prompt)

In [95]:
supervisor = create_tool_calling_agent(llm, tools, supervisor_prompt)

In [96]:
agent21 = create_tool_calling_agent(llm, tools=tools, prompt= greeting_prompt)

In [97]:
# Agent 1: Generally good at math questions
agent1 = create_tool_calling_agent(
    llm=llm,
    tools=[search],
    prompt=math_prompt,
)

In [98]:
agent11 = create_tool_calling_agent(
    llm=llm,
    tools=[search],
    prompt=tool_prompt,
)

In [102]:
from langchain.agents import AgentExecutor
import ast

Supervisor_executor = AgentExecutor(agent = supervisor, tools = [search], verbose = True)

def run_query(query) :
    response = Supervisor_executor.invoke({"input" : query, "Connected_agents" : get_connected_nodes("Supervisor"), "format_instructions" : format_instructions})
    response = ast.literal_eval(response)
    while response["next_agent"] != "None" :
        query = response["next_task"]
        a = locals().get(response["next_agent"])
        agent_executor = AgentExecutor(agent=a, tools=tools, verbose=True)
        response = agent_executor.invoke({"input": query, "Connected_agents" : get_connected_nodes(str(a)), "format_instructions" : format_instructions, "current_output" : response["current_output"]})
    return response["current_output"]


In [103]:
run_query("what is 1+1 ? ")



> Entering new AgentExecutor chain...
```json
{
	"current_output": "2",
	"next_agent": "Agent1",
	"next_task": "Handle basic tasks"
}
```

> Finished chain.


ValueError: malformed node or string: {'input': 'what is 1+1 ? ', 'Connected_agents': [{'Name': 'Agent2', 'Desc': 'Agent doing basic tasks'}, {'Name': 'Agent1', 'Desc': 'Agent to do basic tasks'}], 'format_instructions': 'The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":\n\n```json\n{\n\t"current_output": string  // Output from the current agent that will be passed to the next agent. If there is no next agent, put None\n\t"next_agent": string  // The next agent to handle the request.\n\t"next_task": string  // The task or action for the next agent.\n}\n```', 'output': '```json\n{\n\t"current_output": "2",\n\t"next_agent": "Agent1",\n\t"next_task": "Handle basic tasks"\n}\n```'}